In [ ]:
import os
from dotenv import load_dotenv
from langchain.chat_models import ChatOpenAI
from langchain.schema import BaseOutputParser
from langchain.prompts import ChatPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")
chat = ChatOpenAI(model="gpt-3.5-turbo", openai_api_key=openai_api_key, temperature=0.1, streaming=True, callbacks=[StreamingStdOutCallbackHandler()])

class CommaOutputParser(BaseOutputParser):
    def parse(self, text):
        items = text.strip().split(",")
        return list(map(str.strip, items)) 
        
template = ChatPromptTemplate.from_messages([
    ("system", "당신은 리스트를 생성합니다. 값은 콤마로만 구분해주세요."),
    ("system", "리스트 크기는 {max_length}를 넘을 수 없고, 값은 중복될 수 없습니다."),
    ("human", "{question}")
])

# prompt = template.format_messages(max_length=10, question="국가들을 알려주세요." )
# result = chat.predict_messages(prompt)
# p = CommaOutputParser()
# p.parse(result.content)

chain = template | chat | CommaOutputParser()
chain.invoke({
    "max_length":5,
    "question":"태국의 도시들을 알려주세요."
})

In [ ]:

import os
from dotenv import load_dotenv
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")
chat = ChatOpenAI(model="gpt-3.5-turbo", openai_api_key=openai_api_key, temperature=0.1, streaming=True, callbacks=[StreamingStdOutCallbackHandler()])


chef_prompt = ChatPromptTemplate.from_messages([
    ("system", "당신은 월드클래스 요리사입니다. 당신은 구하기 쉬운 재료로 쉽게 따라할 수 있는 요리 레시피를 만들 수 있습니다."),
    ("human", "저는 {country} 요리를 만들고 싶습니다.")
])
chef_chain = chef_prompt | chat


veg_prompt = ChatPromptTemplate.from_messages([
    ("system", "당신은 월드클래스 채식 요리사입니다. 당신은 전달 받은 레시피를 통해 채식 요리 레시피로 변경할 수 있습니다."),
    ("system", "대체 재료를 명확히 찾을 수 없거나, 레시피를 너무 많이 바꿔야 한다면 할 수 없다고 말해야 합니다."),
    ("human", "{recipe}")
])

veg_chain = veg_prompt | chat
final_chain = {"recipe" : chef_chain} | veg_chain
result = final_chain.invoke({"country":"한국"}).content

print(result)


In [22]:

import os
from dotenv import load_dotenv
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")
chat = ChatOpenAI(
    model="gpt-3.5-turbo", 
    openai_api_key = openai_api_key, 
    temperature = 0.1, 
    streaming = True, 
    callbacks = [
        StreamingStdOutCallbackHandler()
    ]
)

# t = PromptTemplate.from_template("What is the capital of {country}?")
# t.format(country="Korea")

examples = [
    {
        "question": "What do you know about France?",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Italy?",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Greece?",
        "answer": """
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
]

example_prompt = PromptTemplate.from_template("Human: {question}\nAI: {answer}")

prompt = FewShotPromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
    suffix="Human: What do you know about {country}?",
    input_variables=["country"]
)

prompt.format(country="Germany")

chain = prompt | chat

chain.invoke({
    "country":"Korea"
})


AI: 
        I know this:
        Capital: Seoul
        Language: Korean
        Food: Kimchi and Bibimbap
        Currency: South Korean Won

AIMessageChunk(content='AI: \n        I know this:\n        Capital: Seoul\n        Language: Korean\n        Food: Kimchi and Bibimbap\n        Currency: South Korean Won')